In [1]:
# 이전 실습 데이터를 다시 불러와서 검증 세트를 만든다.

import pandas as pd
wine = pd.read_csv('data/wine.csv')

In [2]:
# class열을 타깃으로 하고 나머지열은 특성 배열에 저장한다.
 
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련세트와 테스트 세트를 나눈다. 
 
from sklearn.model_selection import train_test_split
 
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
# 그다음 train_input과 train_target을 훈련세트와 검증 세트로 나눈다.
# train_input의 약 20%를 val_input으로 만든다.
 
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
# 훈련세트와 검증 세트 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
# 모델 훈련하고 평가한다.
 
from sklearn.tree import DecisionTreeClassifier
 
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
 
# 이 모델은 훈련세트에 과대적합이다. 매개변수를 사용해서 모델을 평가해야 한다.
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### 교차 검증

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)

scores

{'fit_time': array([0.0079782 , 0.00801301, 0.00898886, 0.00797057, 0.0079813 ]),
 'score_time': array([0.00099802, 0.00099802, 0.00099063, 0.00099158, 0.00098896]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
import numpy as np
 
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
 
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

### 하이퍼 파라미터 튜닝

In [21]:
from sklearn.model_selection import GridSearchCV
 
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
 
#그리드서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [22]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [23]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [24]:
# 서치로 찾은 최적의 매개변수는 best_params_속성에 저장되어 있다.
# 여기서는 0.0001이 가장 좋은 값으로 선택되었다.
 
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [25]:
# 5번 교차 검증으로 얻은 점수를 출력한다.
 
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [26]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [29]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
         'max_depth' : range(5, 20, 1),
         'min_samples_split' : range(2, 100, 10)
         }

In [30]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [31]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [32]:
print(gs.cv_results_['mean_test_score'])

[0.85780355 0.85799604 0.85799604 ... 0.86126601 0.86165063 0.86357629]


In [33]:
# 랜덤 서치
# 랜덤 서치에는 매개변수 값의 목록을 전달하는 것 아니라 매개변수를 샘플링
# 할 수 있는 확률 분포 객체를 전달한다.
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

array([8, 2, 9, 7, 3, 0, 6, 4, 2, 1])

In [34]:
# 000개의 샘플링해서 각 숫자의 개수를 세어본다.
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 94, 110, 106, 122, 105,  92,  95,  82, 111,  83], dtype=int64))

In [35]:
# uniform도 사용한다
 
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.85174941, 0.68756607, 0.81861999, 0.56553808, 0.43484555,
       0.44927509, 0.9859177 , 0.64940985, 0.56447372, 0.80209825])

In [36]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [37]:
from sklearn.model_selection import RandomizedSearchCV
 
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228CE46D9C8>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228CE46DE48>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228CE458988>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228CE67B188>},
                   random_state=42)

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [40]:
dt = gs.best_estimator_
 
print(dt.score(test_input, test_target))

0.86
